In [2]:
import datetime as dt
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import yfinance as yf
import OptionLib as opt
from datetime import datetime
import datetime
import datetime
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# opt.opt_vol(market_price, spot, strike, r=0.05, time=1, volatility_candidates = np.arange(0.01,4,0.0001))


In [12]:
df = opt.yf_get_chains('JPM')
df # JPM230721C00140000 # JPM230428P00124000

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,optionType,expDate,daysToExp
0,JPM230519C00065000,2023-05-08 19:16:38+00:00,65.0,72.30,68.75,69.35,0.000000,0.000000,1.0,2,2.562504,True,REGULAR,USD,call,2023-05-19 23:59:59,7
1,JPM230519C00070000,2023-05-11 17:40:53+00:00,70.0,66.30,63.80,64.45,0.000000,0.000000,1.0,11,1.734376,True,REGULAR,USD,call,2023-05-19 23:59:59,7
2,JPM230519C00075000,2023-04-04 19:50:05+00:00,75.0,53.15,58.85,60.30,0.000000,0.000000,2.0,0,2.347660,True,REGULAR,USD,call,2023-05-19 23:59:59,7
3,JPM230519C00080000,2023-04-21 15:56:49+00:00,80.0,53.45,53.85,54.45,-7.000000,-11.579818,1.0,4,1.515627,True,REGULAR,USD,call,2023-05-19 23:59:59,7
4,JPM230519C00090000,2023-04-13 18:59:47+00:00,90.0,39.15,43.80,44.45,0.000000,0.000000,1.0,0,1.109379,True,REGULAR,USD,call,2023-05-19 23:59:59,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,JPM251219P00160000,2023-03-23 15:12:58+00:00,160.0,37.14,27.35,29.35,0.000000,0.000000,NaN,2,0.130899,True,REGULAR,USD,put,2025-12-19 23:59:59,952
19,JPM251219P00170000,2023-05-11 19:20:18+00:00,170.0,38.93,39.05,41.05,0.000000,0.000000,2.0,2,0.181313,True,REGULAR,USD,put,2025-12-19 23:59:59,952
20,JPM251219P00190000,2023-05-02 18:47:59+00:00,190.0,51.74,54.55,58.25,0.000000,0.000000,1.0,0,0.175973,True,REGULAR,USD,put,2025-12-19 23:59:59,952
21,JPM251219P00195000,2023-04-21 13:30:18+00:00,195.0,55.30,60.25,63.50,0.000000,0.000000,3.0,3,0.189950,True,REGULAR,USD,put,2025-12-19 23:59:59,952


In [60]:
# opt.add_option(option_chain, df_portfolio, contract, number_of_contracts, spot, position=1,  r=0.05, ticker_name='JPM')
contract_name = 'JPM230526C00133000'
ticker_name = 'JPM'
spot = 130.38
strike = 145

In [31]:
df_portfolio = opt.create_portfolio()
df_portfolio = opt.add_option(option_chain=df, df_portfolio=df_portfolio,
                              contract = contract_name,
                              number_of_contracts=1,
                              spot=spot,
                              position=1,
                              ticker_name=ticker_name)
df_portfolio

,contractSymbol,underlying,expDate,daysToExp,optionType,strike,position,number_of_contracts,delta,delta_pos,DH_quantity
0,JPM230526C00133000,JPM,2023-05-26 23:59:59,14,call,133.0,1,1,0.880694,88.069435,88.069435


# we use it to simulate real data

In [32]:
scenario_underlying = opt.gbm(n_scenarios=1, s_0=120, mu=.002)
option_scenario = opt.gbm(n_scenarios=1, s_0=25, mu=-.002)

In [55]:
history = pd.DataFrame({'underlying_price': [], 'option_price': [],
                        'delta': [], 'delta_pos': [],
                        'DH_quantity': [], 'underlying_needed': [], 'underlying_pos': []})

number_of_steps = scenario_underlying.shape[0]
for step in range(number_of_steps):

    print('Step:', step+1, 'out of', number_of_steps)

    spot = float(scenario_underlying.iloc[step])
    option_market_price = float(option_scenario.iloc[step])
    rebalanced_df = opt.rebalance(df_portfolio=df_portfolio, market_price=option_market_price, spot=spot, ticker_name=ticker_name)

    for contract in list(df_portfolio['contractSymbol']):
        option = df_portfolio[ df_portfolio['contractSymbol'] == contract].reset_index(drop=True).to_dict()
        history.loc[len(history.index)] = [ spot, option_market_price,
                                            option['delta'][0], option['delta_pos'][0],
                                            option['DH_quantity'][0], option['DH_quantity'][0]/spot, -np.sign(option['delta_pos'][0])]

    df_portfolio = rebalanced_df


Step: 1 out of 121
Step: 2 out of 121
Step: 3 out of 121
Step: 4 out of 121
Step: 5 out of 121
Step: 6 out of 121
Step: 7 out of 121
Step: 8 out of 121
Step: 9 out of 121
Step: 10 out of 121
Step: 11 out of 121
Step: 12 out of 121
Step: 13 out of 121
Step: 14 out of 121
Step: 15 out of 121
Step: 16 out of 121
Step: 17 out of 121
Step: 18 out of 121
Step: 19 out of 121
Step: 20 out of 121
Step: 21 out of 121
Step: 22 out of 121
Step: 23 out of 121
Step: 24 out of 121
Step: 25 out of 121
Step: 26 out of 121
Step: 27 out of 121
Step: 28 out of 121
Step: 29 out of 121
Step: 30 out of 121
Step: 31 out of 121
Step: 32 out of 121
Step: 33 out of 121
Step: 34 out of 121
Step: 35 out of 121
Step: 36 out of 121
Step: 37 out of 121
Step: 38 out of 121
Step: 39 out of 121
Step: 40 out of 121
Step: 41 out of 121
Step: 42 out of 121
Step: 43 out of 121
Step: 44 out of 121
Step: 45 out of 121
Step: 46 out of 121
Step: 47 out of 121
Step: 48 out of 121
Step: 49 out of 121
Step: 50 out of 121
Step: 51 

# PnL when buy/sell without an option price or transaction cost, we also assume shares perfectly divisible

In [56]:
history['PnL without Opt'] = (history - history.shift(periods=1))['underlying_price']
history = history.dropna()

history['PnL without Opt'] = history['PnL without Opt'] * history['underlying_needed'] * history['underlying_pos']

In [57]:
history

,underlying_price,option_price,delta,delta_pos,DH_quantity,underlying_needed,underlying_pos,PnL without Opt
1,112.847627,27.505085,1.0,100.0,100.0,0.886151,-1.0,6.338080
2,109.067173,25.266709,1.0,100.0,100.0,0.916866,-1.0,3.466171
3,107.605253,25.266730,1.0,100.0,100.0,0.929323,-1.0,1.358595
4,113.204313,24.697983,1.0,100.0,100.0,0.883359,-1.0,-4.945977
5,113.482322,24.937870,1.0,100.0,100.0,0.881195,-1.0,-0.244981
...,...,...,...,...,...,...,...,...
116,217.002636,17.817412,1.0,100.0,100.0,0.460824,-1.0,1.101010
117,200.560212,18.664307,1.0,100.0,100.0,0.498603,-1.0,8.198248
118,187.363105,17.976231,1.0,100.0,100.0,0.533723,-1.0,7.043600
119,182.522930,17.875612,1.0,100.0,100.0,0.547876,-1.0,2.651818


In [58]:
sum(history['PnL without Opt'])

-28.721819298286405

In [66]:
history['bsm_iv'] = history.apply(lambda x:
                        opt.opt_vol(market_price=x[1],
                                    spot=x[0],
                                    strike=strike,
                                    r=0.05,
                                    time=1, # with real data have column days to expiration and substitute here
                                    type='call'),
                             axis=1)

# We use Bisection method due to the following reasons

The option price with a Heston volatility model depends on the Heston parameters only. That is, the implied volatility parameters does not enter into the
Heston option price formula. Unless there is an analytical formula to compute the implied volatility for a given option price, it is impossible to compute
the implied volatility directly from the Heston parameters. What people usually do is to first compute the European option, from the Heston volatility
process, using either Fourier transformation or Monte Carlo, and then compute the implied volatility using the Newton's method or the bi-section method.

so opt_vol or Newton's method

There is 'in-model' vega and 'out-model' vega. The former is a definition of vega consistent with the model. In stoch vol models such as Heston the correct definition would be indeed to bump $v_0$. The latter is however you want to define vega. Some calculate vega as the BS vega (i.e. wrt to implied vol), others maybe wrt to the variance swap strike, or something else. The reason 'out-of-model' vega is popular is because nobody really believes eg Heston, and out of model vega could make it easier to vega hedge. – Frido Feb 14 at 12:47

https://quant.stackexchange.com/questions/74630/how-to-compute-vega-in-the-heston-model


In [72]:
history = history[:20]

In [85]:
def heston_implied_vol(option_type, option_price, S, K, r, T, q):
    # apply bisection method to get the implied volatility by solving the BSM function
    precision = 0.00001
    upper_vol = 50.0
    max_vol = 50.0
    min_vol = 0.0001
    lower_vol = 0.0001
    iteration = 0

    while 1:
        iteration +=1
        mid_vol = (upper_vol + lower_vol)/2.0

        if option_type == 'call':



            price = opt.Heston(S0=S, strike=K, time=T, r=r, kappa=0.0,
                               theta=0.01,
                               v0=mid_vol**2,
                               lamda=0.0, sigma=0.000001, rho=-0.7).price(0.00001, 100, 10000)[option_type]

            lower_price = opt.Heston(S0=S, strike=K, time=T, r=r, kappa=0.0,
                               theta=0.01,
                               v0=lower_vol**2,
                               lamda=0.0, sigma=0.000001, rho=-0.7).price(0.00001, 100, 10000)[option_type]

            """
            price = bs_call(S,K,T,r,q,mid_vol)
            lower_price = bs_call(S,K,T,r,q,lower_vol)
            """


            if (lower_price - option_price) * (price - option_price) > 0:
                lower_vol = mid_vol
            else:
                upper_vol = mid_vol
            if abs(price - option_price) < precision: break
            if mid_vol > max_vol - 5 :
                mid_vol = 0.0001
                break

        elif option_type == 'put':

            price = opt.Heston(S0=S, strike=K, time=T, r=r, kappa=0.0,
                               theta=0.01,
                               v0=mid_vol**2,
                               lamda=0.0, sigma=0.000001, rho=-0.7).price(0.00001, 100, 10000)[option_type]

            upper_price = opt.Heston(S0=S, strike=K, time=T, r=r, kappa=0.0,
                               theta=0.01,
                               v0=upper_vol**2,
                               lamda=0.0, sigma=0.000001, rho=-0.7).price(0.00001, 100, 10000)[option_type]


            """
            price = bs_put(S,K,T,r,q,mid_vol)
            upper_price = bs_put(S,K,T,r,q,upper_vol)
            """

            if (upper_price - option_price) * (price - option_price) > 0:
                upper_vol = mid_vol
            else:
                lower_vol = mid_vol

            if abs(price - option_price) < precision: break

            if iteration > 100: break

    return mid_vol

print (heston_implied_vol('call', 5.57, 100, 100, 0.05, 1, 0))

0.061087350559234616


In [87]:
history['heston_iv'] = history.apply(lambda x: heston_implied_vol(option_type='call', option_price=x[1],
                    S=x[0],
                   K=strike,
                   r=0.05,
                   T=1, # with real data have column days to expiration and substitute here
                   q=1),
                axis=1)

/var/folders/xd/xm863f553v750zss70sl77sc0000gn/T/ipykernel_1808/3256541917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['heston_iv'] = history.apply(lambda x: heston_implied_vol(option_type='call', option_price=x[1],


### Now we just need to play with parameters such as sigma (vol of vol) and etc., for now it just coincides with bsm results.

In [88]:
history

,underlying_price,option_price,delta,delta_pos,DH_quantity,underlying_needed,underlying_pos,PnL without Opt,bsm_iv,heston_iv
1,112.847627,27.505085,1.0,100.0,100.0,0.886151,-1.0,6.338080,0.7982,0.798160
2,109.067173,25.266709,1.0,100.0,100.0,0.916866,-1.0,3.466171,0.7945,0.794509
3,107.605253,25.266730,1.0,100.0,100.0,0.929323,-1.0,1.358595,0.8129,0.812882
4,113.204313,24.697983,1.0,100.0,100.0,0.883359,-1.0,-4.945977,0.7311,0.731064
5,113.482322,24.937870,1.0,100.0,100.0,0.881195,-1.0,-0.244981,0.7331,0.733065
6,124.983865,26.013391,1.0,100.0,100.0,0.800103,-1.0,-9.202422,0.6219,0.621906
7,135.231983,26.530176,1.0,100.0,100.0,0.739470,-1.0,-7.578176,0.5170,0.516983
8,137.087064,28.214460,1.0,100.0,100.0,0.729463,-1.0,-1.353214,0.5280,0.528027
9,132.047043,28.792209,1.0,100.0,100.0,0.757306,-1.0,3.816837,0.5965,0.596463
10,123.252643,29.567043,1.0,100.0,100.0,0.811342,-1.0,7.135263,0.7150,0.715049


In [ ]:
"""
p1 = SABR(1, 2, 3, 4) # alpha, beta, rho, nu
alpha, rho, nu = p1.calibrate(dataframe) # beta = 0.5 or linear regression or otherwise predicted
def calibrate(self, dataframe, beta=0.5, T=11/252, f=193.81,
                  alpha_rho_nu_guess = np.array([.03, -.5, .4]), method='trust-constr', setparams=True)

here is df with impliedVolatility needed
we got alpha, beta, rho, nu
then substitute to the hagan_iv(K, T, f, alpha, beta, rho, nu)

maybe calc alpha, beta, rho, nu once anf in hagan_iv change T, f only
"""